# Introducing logistic regression

A logistic regression "squashes" a linear expression into a 0-1 range using a logistic, or sigmoidal, function.  That looks like this:

In [ ]:
import numpy as np
lim = 6
t = np.linspace(-lim, lim, 100)
sig = 1 / (1 + np.exp(-t))

plt.figure(figsize=(8, 3))
plt.plot([-lim, lim], [0, 0], "k-")
plt.plot([-lim, lim], [0.5, 0.5], "k:")
plt.plot([-lim, lim], [1, 1], "k:")
plt.plot([0, 0], [-1.1, 1.1], "k-")
plt.plot(t, sig, "b-", linewidth=2, label=r"$\sigma(t) = \dfrac{1}{1 + e^{-t}}$")
plt.xlabel("t")
plt.legend(loc="upper left")
plt.axis([-lim, lim, -0.1, 1.1])
plt.gca().set_yticks([0, 0.25, 0.5, 0.75, 1])
plt.grid()
plt.show()

We'll demonstrate a logistic regression first by using the famous 'iris' dataset.

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)
list(iris)

We only care about data and target right now.  First, let's read the dataset description.

In [ ]:
print(iris.DESCR)

In [ ]:
iris.data.head(3)

In [ ]:
iris.target_names

In [ ]:
iris.target.head(3)

Note, by conventions the "targets" correspond t indices in the target names array.  So, the above are all 'setosa'.  As the slmplest form of a logistic reggression is binary classifier, let's focus just on predicting "virginica". We'll do that by relabeling our "targets".  By convention, we often use the variable `X` to refer to data, and `y` to refer to targets.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = iris.data.values
y = iris.target_names[iris.target] == 'virginica'

Now we create a train / test split, and then train the classifier.  See comments in code!

In [ ]:
# train test split creates four groups.  Random state is set here for replicability purposes, and is optional
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Create the machine learning model - this is typically how we do things in SciKit learn
log_reg = LogisticRegression(random_state=42)

# Train the clasifier
log_reg.fit(X_train, y_train)

Ok, great!  Now we can examine performance.  Typically, we'd do this by predicting classes, and the comparing the outcome.  Like this.

In [ ]:
# Here are the predictions
y_pred = log_reg.predict(X_test)

We can use any number of scoring routines from scikit learn to test our results. But let's do it manually first (this is just accuracy).


In [ ]:
n_correct = sum(y_pred == y_test)
print(n_correct / len(y_pred))

## Exercise 1:

Try adding some noise to your data and see how your accuracy changes.  Note that you can add noise to a matrix in numpy like this:

In [ ]:
sample = np.ones((10,10))
print("Original matrix")
print(sample)

# 0 is the mean, .3 is the standard deviation
# The last parameter is the shape of the matrix we want to retrieve
noise = np.random.normal(0,.3,sample.shape)
noisy_sample = sample + noise
print("\n\nNoisy matrix")
print(noisy_sample)


If you're feeling ambitious, create a loop and graph your accuracy across noise levels!  You'll want a function for this.

# Evaluating classifiers

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False)

In [ ]:
import pprint
pp = pprint.PrettyPrinter(width=120)
pp.pprint(mnist.DESCR)

In [ ]:
mnist.keys()  # extra code – we only use data and target in this notebook

Note, data and target exist for most sklearn datasets.  Data is a matrix of features, and target is a vector of labels.

In [ ]:
X, y = mnist.data, mnist.target
X

If you wanted to see these as a DataFrame, you could do this:

In [ ]:
import pandas as pd
df = pd.DataFrame(mnist.data)
df["target"] = mnist.target
df

But that's not really necessary here.  We'll just use these things as matrices and arrays.  Let's look at the shape here.

In [ ]:
print(f"X shape = {X.shape}")
print(f"y shape = {y.shape}")
print(f"Is number of rows equal? {X.shape[0] == y.shape[0]}")

Where does the 784 come from?  Read the docs above!

In [ ]:
28 * 28

Let's take a look at one of these images

In [ ]:
import matplotlib.pyplot as plt

# Define function to plot a single digit
def plot_digit(image_data):
    image = image_data.reshape(28, 28)
    plt.imshow(image, cmap="binary")
    plt.axis("off")

some_digit = X[0]
plot_digit(some_digit)
plt.show()

What is it?

In [ ]:
y[0]

Here's another sample:

In [ ]:
plt.figure(figsize=(9, 9))
for idx, image_data in enumerate(X[:100]):
    plt.subplot(10, 10, idx + 1)
    plot_digit(image_data)
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

### Training the classfier

Once again, since we're using logistic regression, let's just go with predicting '5s'.

In [ ]:
from sklearn.preprocessing import StandardScaler
# Setting up the data here
# Though it's not really necessary here (MINST variables are all roughly the same) it's often useful to z-score
# your data.  Especially important for logistic regression
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# Arbitratily taking the first 60000 rows here
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

y_train_5 = (y_train == '5')  # True for all 5s, False for all other digits
y_test_5 = (y_test == '5')

In [ ]:
# I'm going to use the SGDClassifier here - it's a lot like a logistic regression with an "sag" solver,
# but is a lot faster with larger data sets

from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

In [ ]:
# Recall we already checked that X[0] is a five
some_digit = X[0]

# Note that predict expect an array of values, so we tuck this into an array before testing.
sgd_clf.predict([some_digit])

## Performance measures

### Measuring accuracy with cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score

# Scikit learn makes it easy to use cross validation with simple measures
# CV is the number of folds
cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")


# Exercise 2

Go ahead and dig into the docs to use the following scoring methods, reporting the average for each (note `np.mean` will return average over an array)

1. Precision
2. Recall
3. F1 scores

If you want more control over your classifier, use the `KFold` and related classes.

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.metrics import f1_score

skfolds = StratifiedKFold(n_splits=3)  # add shuffle=True if the dataset is not
                                       # already shuffled
for train_index, test_index in skfolds.split(X_train, y_train_5):
    # It's a good idea to use a fresh, untrained model each time you run on new data
    # The "clone" command does that, but simplifies things by copying other parameters
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = y_train_5[train_index]
    X_test_fold = X_train[test_index]
    y_test_fold = y_train_5[test_index]

    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

Seems quite good!  But also always useful to compare to a naive classifer.  The `DummyClassifier` is one such instance.

In [ ]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier()
cross_val_score(dummy_clf, X_train, y_train_5, cv=3, scoring="accuracy")


## Confusion Matrix

Instead of `cross_val_score` we can just use `cross_val_predict` to get the raw predictions - sklearn takes care of compiling our results so they are easy to process

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)
y_train_pred

In [ ]:
y_train_5.shape == y_train_pred.shape

In [ ]:
# You've hopefully already found the sklearn metrics library
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_train_5, y_train_pred)
cm

In [ ]:
# Just to get a sense of things, see what happens if we had a perfect predictor

y_train_perfect_predictions = y_train_5  # pretend we reached perfection
confusion_matrix(y_train_5, y_train_perfect_predictions)


## Precision and Recall

In [ ]:
from sklearn.metrics import precision_score, recall_score

precision_score(y_train_5, y_train_pred)  # == 3530 / (687 + 3530)

In [ ]:
# Same computation, using the confusion matrix above
# TP / (FP + TP)
cm[1, 1] / (cm[0, 1] + cm[1, 1])

In [ ]:
recall_score(y_train_5, y_train_pred)  # == 3530 / (1891 + 3530)

In [ ]:
# TP / (FN + TP)
cm[1, 1] / (cm[1, 0] + cm[1, 1])

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_train_5, y_train_pred)

In [ ]:
# Calculating by hand
cm[1, 1] / (cm[1, 1] + (cm[1, 0] + cm[0, 1]) / 2)

## Precision/ Recall Trade-off

In [ ]:
# The "decision_function" method returns the raw value, of the predictor, which is then 
# thresholded to achieve an outcome

y_scores = sgd_clf.decision_function([some_digit])
y_scores

In [ ]:
# We can plug this directly into cross_val_predict to get the scores across all of our data
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3,
                             method="decision_function")

In [ ]:
# Scikit learn gives us a really nice way to look at this!
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
threshold = 3000
precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)
plt.figure(figsize=(8, 4))  # extra code – it's not needed, just formatting
plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
plt.vlines(threshold, 0, 1.0, "k", "dotted", label="threshold")

idx = (thresholds >= threshold).argmax()  # first index ≥ threshold
plt.plot(thresholds[idx], precisions[idx], "bo")
plt.plot(thresholds[idx], recalls[idx], "go")
plt.axis([-50000, 50000, 0, 1])
plt.grid()
plt.xlabel("Threshold")
plt.legend(loc="center right")


plt.show()

In [ ]:
# We can graph these two together:

import matplotlib.patches as patches  # extra code – for the curved arrow

plt.figure(figsize=(6, 5))  # extra code – not needed, just formatting

plt.plot(recalls, precisions, linewidth=2, label="Precision/Recall curve")

plt.plot([recalls[idx], recalls[idx]], [0., precisions[idx]], "k:")
plt.plot([0.0, recalls[idx]], [precisions[idx], precisions[idx]], "k:")
plt.plot([recalls[idx]], [precisions[idx]], "ko",
         label="Point at threshold 3,000")
plt.gca().add_patch(patches.FancyArrowPatch(
    (0.79, 0.60), (0.61, 0.78),
    connectionstyle="arc3,rad=.2",
    arrowstyle="Simple, tail_width=1.5, head_width=8, head_length=10",
    color="#444444"))
plt.text(0.56, 0.62, "Higher\nthreshold", color="#333333")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.axis([0, 1, 0, 1])
plt.grid()
plt.legend(loc="lower left")

plt.show()

In [ ]:
# With this analysis, we can arbitrarily obtain a threshold to achieve a given level of precision or recall:

idx_for_90_precision = (precisions >= 0.90).argmax()
threshold_for_90_precision = thresholds[idx_for_90_precision]
threshold_for_90_precision

# Exercise 3

1. What is the precision score at a threshold of 90?
2. What is recall at this threshold?  F1 score?
3. Can you do the same thing, except optimizing to achieve recall > .6?

## ROC Curves

In [ ]:
# SciKit learn also gives us ROC curves for free

from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)

idx_for_threshold_at_90 = (thresholds <= threshold_for_90_precision).argmax()
tpr_90, fpr_90 = tpr[idx_for_threshold_at_90], fpr[idx_for_threshold_at_90]


plt.plot(fpr, tpr, linewidth=2, label="ROC curve")
plt.plot([0, 1], [0, 1], 'k:', label="Random classifier's ROC curve")
plt.plot([fpr_90], [tpr_90], "ko", label="Threshold for 90% precision")

# just beautifies the figure
plt.gca().add_patch(patches.FancyArrowPatch(
    (0.20, 0.89), (0.07, 0.70),
    connectionstyle="arc3,rad=.4",
    arrowstyle="Simple, tail_width=1.5, head_width=8, head_length=10",
    color="#444444"))
plt.text(0.12, 0.71, "Higher\nthreshold", color="#333333")
plt.xlabel('False Positive Rate (Fall-Out)')
plt.ylabel('True Positive Rate (Recall)')
plt.grid()
plt.axis([0, 1, 0, 1])
plt.legend(loc="lower right", fontsize=13)

plt.show()